In [4]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.io import wavfile
import time

In [6]:
main_path = 'D:/rech/VeriRu/VeriRU/'
folder = ['001','002','003','004','005','006','007','008','009','010','011','012','013','014','015','016','017','018','019','020','021','022','023','024','025','026','027','028','029','030']
path = main_path+folder[0]+'/'
curr_names = pd.read_csv(path+folder[0]+'.txt',delimiter=' ',header=None)[0]
curr_names = path+curr_names
names = pd.DataFrame(curr_names)
for i in range(1,30):
    path = main_path+folder[i]+'/'
    curr_names = pd.read_csv(path+folder[i]+'.txt',delimiter=' ',header=None)[0]
    curr_names = path+curr_names
    names = pd.concat([names,curr_names])
spisok2=list(names[0].values)
labels = [string[1] for string in names[0].str.split('_')]

In [10]:
path_out = 'D:/rech/VeriRu/words/'

for i in range(len(spisok2)):
    
    fs, data = wavfile.read(spisok2[i])

    window=500 # window length 
    E1 = []
    E2 = []
    N = []
    for j in range(0,len(data),window):

        if j<len(data)-window:
            viborka = data[j:j+window]
        else:
            viborka = data[j:j+window//2]


        sred = np.mean(viborka)
        disp = np.var(viborka)
        stand = np.std(viborka)

        n=0
        k=1
        e1=0
        e2=0
        
        # short-term energy
        for m in range(0,len(viborka)): 
            e1 = e1 + viborka[m]**2; 
            e2 = e2 + np.abs(viborka[m]);
        E1 = np.hstack((E1,e1))
        E2 = np.hstack((E2,e2))

        # average zero crossings
        for m in range(1,len(viborka)):
            n = n + np.abs(np.sign(viborka[m]) - np.sign(viborka[m-1])/2)
        N=np.hstack((N,n));




        # using moving average for smoothing the data
        E1_filt = np.empty([len(N)])
        E2_filt = np.empty([len(N)])
        N_filt  = np.empty([len(N)])

        E1_filt[0]= E1[0]
        E2_filt[0]= E2[0]
        N_filt[0] = N[0]
        k=3.15;
        for x in range(0,len(N)-1):
            N_filt[x+1]  = ((k-1)*N_filt[x]  + N[x+1])/k
            E1_filt[x+1] = ((k-1)*E1_filt[x] + E1[x+1])/k
            E2_filt[x+1] = ((k-1)*E2_filt[x] + E2[x+1])/k

        # finging start and end of the word fragment in signal with Rabiner-Schafer algorithm

    b = 30 # number of standard deviations for inner interval 
    c = 5 # number of standard deviations for outer interval 
    d = 8 # number of frames for collecting statistacial information
    p = 3 # number of standard deviations for interval in average zero crossings threshold

    sred_E1 = np.mean(E1_filt[0:d])
    disp_E1 = np.var(E1_filt[0:d])
    stand_E1 = np.std(E1_filt[0:d])
    interval_E1 = sred_E1 + b*stand_E1
    outer_interval_E1 = sred_E1 + c*stand_E1

    sred_E2 = np.mean(E2_filt[0:d])
    disp_E2 = np.var(E2_filt[0:d])
    stand_E2 = np.std(E2_filt[0:d])
    interval_E2 = sred_E2 + b*stand_E2
    outer_interval_E2 = sred_E2 + c*stand_E2

    sred_N = np.mean(N_filt[0:d])
    disp_N = np.var(N_filt[0:d])
    stand_N = np.std(N_filt[0:d])
    interval_N = sred_N + p*stand_N
    # outer_interval_N = sred_E2 + c*stand_N

    # inner borders
    flag = 0 
    for ii in range(len(E2_filt)):
        if ~flag & (E2_filt[ii] > interval_E2):
            flag = 1
            point_1a = ii
        if flag & (E2_filt[ii] < interval_E2):
            point_1b = ii
            break
    # left outer border
    for jj in range(point_1a,0,-1):
        if E2_filt[jj] < outer_interval_E2:
            point_2a=jj
            break

    # right outer border
    for jj in range(point_1b,len(E2_filt)):
        if E2_filt[jj] < outer_interval_E2:
            point_2b=jj
            break

    # left final border(based on average zero crossings)        
    cnt = 0
    point_3a = point_2a
    for kk in range(point_2a,0,-1):
        cnt+=1
        if N_filt[kk]<interval_N:
            point_3a = kk
            break
        if cnt==25:
            break

    cnt = 0
    point_3b = point_2b
    for kk in range(point_2b,len(N_filt)):
        cnt+=1
        if N_filt[kk]<interval_N:
            point_3b = kk

            break
        if cnt==25:
            break

    # file writing
    word_start = window * point_3a
    word_end = window * point_3b
    word=data[word_start:word_end]
    if len(word)<8000:
        print('Меньше 8к',i)
        if len(word)<5000:
            print('Меньше 5к',i)
    wavfile.write(path_out+str(i)+'.wav',fs, word)

D:\Progi\Anaconda\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: overflow encountered in int_scalars


Плохие: 469, 614,1619, 3210, 3398, 4086, 5518, 5546, 5990, 5998, 6812, 7124, 7135, 8001, 8689, 8722, 8733, 8771, 9216, 10533

Сохраняем лейблы, удалив плохие. Из самих данных удалим потом, в скрипте мел-кепстров.

In [50]:
bad = np.array([469, 614,1619, 3210, 3398, 4086, 5518, 5546, 5990, 5998, 6812, 7124, 7135, 8001, 8689, 8722, 8733, 8771, 9216, 10533])-1
labels_cleared = pd.Series(labels,name='label').drop(bad)
labels_cleared.to_csv('D:/rech/VeriRu/words/labels.csv',index_label='idx')